In [4]:
import json

import time
from NER import preprocess
TRAINING_SET = "/home/neeserg/Dropbox/Webtext/Data/train.json"

# Index Sentences as Document

We used this at first, and used this file to write the SNLI file. But did not used the updated document based indexing to train the model, so do not have a code base to write it based on the file.

In [ ]:
import xapian
import time
DATA_FILEPATH = ##path of wiki file
DATABASE_FILEPATH = ##path of xapian database
start = time.time()
db = xapian.WritableDatabase(DATABASE_FILEPATH, xapian.DB_CREATE_OR_OPEN)

termgenerator = xapian.TermGenerator()
termgenerator.set_stemmer(xapian.Stem("en"))

for i in range(1,110):
    st = "wiki-{:03d}.txt".format(i)
    cyclestart = time.time()
    print(st)
    j = 0
    with open(DATA_FILEPATH + st) as file:
       
        #Create the databse
        for line in file:
            words = line.split(' ')
            
            #extract the title from the id

            id1 = words[0]
            title = id1.split('_')
            title = ' '.join(title)
            title = title.split('-')
            title = ' '.join(title)

            #check fact number is given and create doc ID
            id2 = words[1]
            if not id2.isdigit():
                continue
            docId = u"Q"+id1 + "_~s~_"+id2

            #create main fact refference
            text = ' '.join(words[2:])
            doc = xapian.Document()
            termgenerator.set_document(doc)
            termgenerator.index_text(text)
            termgenerator.index_text(title, 1,'S')
            if (j%10000 == 0):
                print(docId, text)

            doc.set_data(text)
            doc.add_boolean_term(docId)
            db.replace_document(docId, doc)
            j+=1
    print(st, time.time() - cyclestart)

print("This program took this many seconds: ",time.time() - start )



###This is document based indexing that we used to make our prediction

# ##export XAPIAN_FLUSH_THRESHHOLD=200000; python index_alldoc.py
# import xapian
# import time
# DATA_FILEPATH = "/Users/neesergparajuli/Dropbox/Webtext/Data/wiki-pages-text/"
# DATABASE_FILEPATH = "/Users/neesergparajuli/Dropbox/Webtext/Data/XxapianDatabase4"
# start = time.time()
# db = xapian.WritableDatabase(DATABASE_FILEPATH, xapian.DB_CREATE_OR_OPEN)

# termgenerator = xapian.TermGenerator()
# termgenerator.set_stemmer(xapian.Stem("en"))
# text = ''
# content = ''
# prev_docid = ''
# j = 0
# first_done = False
# for i in range(1,110):
#     st = "wiki-{:03d}.txt".format(i)
#     cyclestart = time.time()
#     print(st)
#     with open(DATA_FILEPATH + st) as file:
       
#         #Create the databse
#         for line in file:
#             words = line.split(' ')
            
#             #extract the title from the id

#             id1 = words[0]
#             docId = u"Q"+id1 + "_~s~_"
#             id2 = words[1]
#             if not id2.isdigit():
#                 continue
            
            
#             if(docId != prev_docid and first_done):
#                 doc = xapian.Document()
#                 termgenerator.set_document(doc)
#                 #termgenerator.index_text(content)
#                 termgenerator.index_text(title, 1,'S')
#                 doc.set_data(text)
#                 doc.add_boolean_term(prev_docid)
#                 db.replace_document(prev_docid, doc)
                
#                 if (j%10000 == 0):
#                     print(prev_docid +"--------" + text)
#                     print(content)
#                     print(j)
#                 text = ''
#                 content = ''
#                 j+=1

#             ############# update document information
#             title = id1.split('_')
#             title = ' '.join(title)
#             content += ' '.join(words[2:])
#             text += '*_*' + id2 + ' '.join(words[2:])
#             prev_docid = docId
#             first_done = True

#     print(st, time.time() - cyclestart)

# print("This program took this many seconds: ",time.time() - start )




# Preprocessing and querying the 

In [ ]:
import nltk
from nltk.stem import *
import spacy
nlp = spacy.load('en_core_web_sm')
##extracting named entities
def preprocess(sent):
    sent = nlp(sent)
    query = []
    for ent in sent.noun_chunks:
        if ent.root.dep_ == "nsubj":
            query.append("title:" + '"' + ent.text + '"')
    for ent in sent.ents:
        query.append("text:" + '"' + ent.text + '"')
    for token in sent:
        if token.tag_ == "NN" :
            query.append("text:" + token.text)
        elif token.pos_ == "PROPN":
            query.append("text:" + token.text)
            
    return query


In [ ]:
db = xapian.Database(DATABASE)

def get_unique_id(terms):
    for term in terms:
        if "_~s~_" in term:
            ID = term.split("_~s~_" )
            sentenceId  = int(ID[1])
            docId = ID[0][1:]
            return (docId, sentenceId)
    
    print("wtdf")
    return ("wtf",1)


def my_own_queryparser(fact):
    new_fact = ''
    for f in fact:
        new_fact += f + ' '
    return new_fact

    # initiate stemmer for query
def query_fact_normally(fact, k_matches):
   
    queryparser = xapian.QueryParser()
    queryparser.set_stemmer(xapian.Stem("en"))
    queryparser.add_prefix("text", "")
    queryparser.add_prefix("title", "S")
    query = queryparser.parse_query(my_own_queryparser(fact))
    enquire = xapian.Enquire(db)
    # enquire.set_weighting_scheme(xapian.TfIdfWeight())
    enquire.set_query(query)
    return list(match.document.get_data().decode("utf8") for match in enquire.get_mset(0,k_matches))

def query_id(abool_term):
    enquire = xapian.Enquire(db)
    bool_term =  xapian.Query(abool_term)
    enquire.set_query(bool_term)
    matches = [match.document.get_data().decode("utf8") for match in enquire.get_mset(0,8)]
    if len(matches)>1:
        print(matches)
    if len(matches) ==0:
        print(abool_term + "doesnt exist")
        return None
    sentence = matches[0]
    return sentence

In [5]:
with open(TRAINING_SET) as file:
    data = json.load(file)
def get_doc_id(evidence_list):
    doc_id_ls = []
    for evidence in evidence_list:
        doc_id = evidence[0]
        sent_num = evidence[1]
        idd = "Q" + doc_id + "_~s~_"+ str(sent_num)
        doc_id_ls.append(idd)
    return doc_id_ls

In [6]:
def write(ev):
    with open("train3.jsonl", "a") as file:
        for e in ev:
            json.dump(e, file)
            file.write('\n')

In [8]:
import random
m = 0
ev = []
j=0
qqq =0

for obj in data:
    label = data[obj]["label"]
    sentence2 = data[obj]["claim"]
    if label == "NOT ENOUGH INFO":
        evidence_set = list(query_fact_normally(preprocess(sentence2),5))
    else:
        evidence_set = list(set(query_id(e) for e in get_doc_id(data[obj]["evidence"])))
    m+=1
    for e in evidence_set:
        premise = e
        if sentence2 and premise:
            ev.append({"sentence2": sentence2, "sentence1": premise, "gold_label": label})
    if (m+1)%1000 ==0:
        if len(evidence_set)>0:
            print(m,evidence_set[0], label, sentence2)
        write(ev)
        ev = []

if ev:  
    write(ev)

QJosé_María_Chacón_~s~_0doesnt exist
QRégine_Chassagne_~s~_0doesnt exist
QChris_Pérez_~s~_0doesnt exist
QChris_Pérez_~s~_24doesnt exist
QBeyoncé_~s~_15doesnt exist
QBeyoncé_~s~_16doesnt exist
QFrédéric_Auguste_Bartholdi_~s~_0doesnt exist
999 Between 2009 and 2011 , Gomez starred in films such as Princess Protection Program , Ramona and Beezus , and Monte Carlo , and took on a more mature role in Spring Breakers -LRB- 2013 -RRB- .
 SUPPORTS Selena Gomez starred in the 2013 film Spring Breakers.
QBjörn_Borg_~s~_0doesnt exist
QMilton_Núñez_-LRB-boxer-RRB-_~s~_1doesnt exist
QWilfredo_Gómez_~s~_0doesnt exist
1999 Jacqueline Marie `` Jackie '' Evancho -LRB- -LSB- iːˈvæŋkoʊ -RSB- ; born April 9 , 2000 -RRB- is an American classical crossover singer who gained wide recognition at an early age and , since 2009 , has issued an EP and seven albums , including a platinum and gold album and three Billboard 200 top 10 debuts .
 SUPPORTS Jackie Evancho has a middle name.
QCitadelle_Lafe

14999 He won his first Primetime Emmy Award for his role in American Horror Story : Asylum -LRB- 2012 -RRB- and a Canadian Screen Award for his role in Still Mine -LRB- 2013 -RRB- .
 SUPPORTS James Cromwell won a Canadian Screen Award for his role in Still Mine.
QFrédéric_Auguste_Bartholdi_~s~_0doesnt exist
QStellan_Skarsgård_~s~_0doesnt exist
QJosé_Aldo_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QB'Day_-LRB-Beyoncé_album-RRB-_~s~_0doesnt exist
QBeyoncé_~s~_11doesnt exist
Q4_-LRB-Beyoncé_album-RRB-_~s~_0doesnt exist
QBeyoncé_~s~_10doesnt exist
QBeyoncé_-LRB-album-RRB-_~s~_0doesnt exist
QBeyoncé_~s~_9doesnt exist
QBeyoncé_~s~_3doesnt exist
QLemonade_-LRB-Beyoncé_album-RRB-_~s~_0doesnt exist
QBeyoncé_~s~_1doesnt exist
QBeyoncé_~s~_0doesnt exist
QBeyoncé_~s~_6doesnt exist
QSophia_Abrahão_~s~_0doesnt exist
QĀtman_-LRB-Hinduism-RRB

QFrançois_de_Belleforest_~s~_9doesnt exist
QThénardiers_~s~_0doesnt exist
QNight_Attack_at_Târgovişte_~s~_0doesnt exist
QMichael_Peña_~s~_0doesnt exist
QBjörn_Ulvaeus_~s~_0doesnt exist
QStefan_Kapičić_~s~_0doesnt exist
QDaniela_Hantuchová_~s~_7doesnt exist
QDaniela_Hantuchová_~s~_9doesnt exist
QDaniela_Hantuchová_~s~_14doesnt exist
QDaniela_Hantuchová_~s~_3doesnt exist
QThénardiers_~s~_0doesnt exist
QFrédéric_Auguste_Bartholdi_~s~_0doesnt exist
QZlatan_Ibrahimović_~s~_10doesnt exist
QGrégory_Levasseur_~s~_0doesnt exist
QBeyoncé_~s~_7doesnt exist
28999 He has also won MTV , ASCAP , American Music awards , three Ivor Novello Awards for song composition and has been nominated five times for Golden Globe Awards and three times for Academy Awards for his songwriting for films .
 SUPPORTS Bryan Adams has won awards.
QThénardiers_~s~_0doesnt exist
QThénardiers_~s~_6doesnt exist
QThénardiers_~s~_7doesnt exist
QJulio_Cortázar_~s~_0doesnt exist
QCría_Cuervos_~s~_0doesnt ex

QFrançois_de_Belleforest_~s~_9doesnt exist
QJosé_María_Chacón_~s~_1doesnt exist
QRóisín_Murphy_~s~_0doesnt exist
QGyőr_~s~_1doesnt exist
QThérèse_Raquin_~s~_0doesnt exist
41999 To the Bone is a 2017 American drama film , written and directed by Marti Noxon .
 REFUTES To the Bone is a restaurant.
QZlatan_Ibrahimović_~s~_8doesnt exist
QZlatan_Ibrahimović_~s~_10doesnt exist
QZlatan_Ibrahimović_~s~_7doesnt exist
QZlatan_Ibrahimović_~s~_6doesnt exist
QZlatan_Ibrahimović_~s~_12doesnt exist
QZlatan_Ibrahimović_~s~_25doesnt exist
QZlatan_Ibrahimović_~s~_20doesnt exist
QZlatan_Ibrahimović_~s~_1doesnt exist
QMalmö_FF_~s~_0doesnt exist
QZlatan_Ibrahimović_~s~_0doesnt exist
QY_Tu_Mamá_También_~s~_4doesnt exist
QJosé_Aldo_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QFrançois_de_Belleforest_~s~_15doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QNight_Attack_at_Târgovişte_~s~_0doesnt exist
QJulio_Cortázar

QSophia_Abrahão_~s~_0doesnt exist
QSofía_Vergara_~s~_0doesnt exist
QGisele_Bündchen_~s~_17doesnt exist
52999 Basic Instinct is a 1992 neo-noir erotic thriller film directed by Paul Verhoeven and written by Joe Eszterhas , and starring Michael Douglas and Sharon Stone .
 REFUTES Basic Instinct was released in 1988.
QRégine_Chassagne_~s~_1doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QHenri_Poincaré_~s~_0doesnt exist
QÉomer_~s~_0doesnt exist
QSofía_Vergara_~s~_11doesnt exist
QPřemyslid_dynasty_~s~_0doesnt exist
QMedellín_Cartel_~s~_0doesnt exist
53999 The Burundi Tribune is a newspaper published in Bujumbura , Burundi .
 NOT ENOUGH INFO Burundi has a humid climate.
QThénardiers_~s~_0doesnt exist
QGrégory_Levasseur_~s~_0doesnt exist
QGrégory_Levasseur_~s~_0doesnt exist
QThénardiers_~s~_0doesnt exist
QThénardiers_~s~_8doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QSofía_Vergara_~s~_0doesnt exist
QSofía_Vergara_~

...

80999 Faith Renée Evans -LRB- born June 10 , 1973 -RRB- is an American singer-songwriter .
 NOT ENOUGH INFO Faith Evans is a director.
QPenélope_Cruz_~s~_0doesnt exist
QBjörn_Ulvaeus_~s~_0doesnt exist
81999 Jed Mercurio -LRB- born Gerald Gary Mercurio in 1966 -RRB- is a British television writer , producer , director and novelist .
 SUPPORTS Line of Duty was created by a person.
QJosé_María_Chacón_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QZlatan_Ibrahimović_~s~_7doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QTré_Cool_~s~_0doesnt exist
82999 As of March 2017 , Wikipedia has about forty thousand high-quality articles known as Featured Articles and Good Articles that cover vital topics .
 REFUTES Wikipedia doesn't have any articles.
QSofía_Vergara_~s~_0doesnt exist
QMusée_National_d'Art_Moderne_~s~_1doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_2doesnt exist
QCitadelle_Laferrière_~s~_1doesnt exist
QFrançois_de_Bell

QHenri_Poincaré_~s~_0doesnt exist
QBjörk_~s~_16doesnt exist
QBjörk_~s~_16doesnt exist
QChristine_Daaé_~s~_0doesnt exist
QAs_Máscaras_~s~_0doesnt exist
96999 Evan Rachel Wood -LRB- born September 7 , 1987 -RRB- is an American actress and musician .
 REFUTES Evan Rachel Wood was born in December.
QHenri_Poincaré_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QZlatan_Ibrahimović_~s~_11doesnt exist
QZlatan_Ibrahimović_~s~_10doesnt exist
QParis_Métro_~s~_8doesnt exist
QMusée_d'Orsay_~s~_5doesnt exist
97999 The English word dragon and Latin word draco derives from Greek δράκων -LRB- drákōn -RRB- , `` dragon , serpent of huge size , water-snake '' .
 REFUTES Dragon is from a Greek word meaning "death."
QCopa_América_~s~_0doesnt exist
QCopa_América_~s~_1doesnt exist
Q2011_Copa_América_~s~_0doesnt exist
QMusée_National_d'Art_Moderne_~s~_1doesnt exist
QMusée_National_d'Art_Moderne_~s~_0doesnt exist
QMusée_National_d'Art_Moderne_~s~_13doesnt exist
QMusée_National_d

QTré_Cool_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QCharlotte_Brontë_~s~_0doesnt exist
QBeyoncé_~s~_7doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QTré_Cool_~s~_0doesnt exist
QThénardiers_~s~_6doesnt exist
QThénardiers_~s~_0doesnt exist
QRégine_Chassagne_~s~_1doesnt exist
QRégine_Chassagne_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_2doesnt exist
QCitadelle_Laferrière_~s~_1doesnt exist
QSofía_Vergara_~s~_0doesnt exist
QRecep_Tayyip_Erdoğan_~s~_0doesnt exist
112999 Voters were also alienated by Davis 's record-breaking fundraising efforts and negative campaigning .
 NOT ENOUGH INFO Gray Davis was a senator.
QBjörn_Ulvaeus_~s~_0doesnt exist
QYolanda_Saldívar_~s~_0doesnt exist
QSophia_Abrahão_~s~_0doesnt exist
QNight_Attack_at_Târgovişte_~s~_0doesnt exist
113999 It was recorded by McEnery in 1941 .
 NOT ENOUGH INFO Amelia Earhart's mother was Mary.
QTré_Co

QPeer_Åström_~s~_1doesnt exist
QPeer_Åström_~s~_2doesnt exist
QPeer_Åström_~s~_0doesnt exist
QGrégory_Levasseur_~s~_0doesnt exist
QCafé_Society_-LRB-film-RRB-_~s~_0doesnt exist
QDon_Balón_Award_~s~_0doesnt exist
QTrophées_UNFP_du_football_~s~_0doesnt exist
QJosé_Aldo_~s~_4doesnt exist
QNight_Attack_at_Târgovişte_~s~_0doesnt exist
QÉdith_Piaf_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
127999 After various modeling work , he made his acting debut as Leo du Pres on the ABC daytime soap opera All My Children and later starred as Danny McCoy on NBC 's Las Vegas .
 SUPPORTS Josh Duhamel is an American male.
QGrégory_Levasseur_~s~_0doesnt exist
128999 He turned 100 on December 9 , 2016 .
 NOT ENOUGH INFO Kirk Douglas was born on December 9th, 1916 in a log cabin.
QBeyoncé_~s~_19doesnt exist
QSophia_Abrahão_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QFrançois_de_Belleforest_~s~_9doesnt exist
QStefan_K

QRégine_Chassagne_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
Q2014_Thai_coup_d'état_~s~_1doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QCitadelle_Laferrière_~s~_2doesnt exist
QJosé_María_Chacón_~s~_1doesnt exist
QCédula_de_identidad_~s~_0doesnt exist
QPenélope_Cruz_~s~_0doesnt exist
QBjörk_~s~_16doesnt exist
QCitadelle_Laferrière_~s~_0doesnt exist
QMusée_National_d'Art_Moderne_~s~_1doesnt exist
QJulio_Cortázar_~s~_0doesnt exist
QThénardiers_~s~_0doesnt exist
141999 In 2017 , Banks will replace Nick Cannon as host of America 's Got Talent for its 12th season .
 REFUTES Tyra Banks presented America's Got Talent for its first twenty-two seasons.
QĀtman_-LRB-Hinduism-RRB-_~s~_0doesnt exist
QĀtman_-LRB-Hinduism-RRB-_~s~_1doesnt exist
QĀtman_-LRB-Hinduism-RRB-_~s~_0doesnt exist
QĀtman_-LRB-Hinduism-RRB-_~s~_5doesnt exist
QĀtman_-LRB-Hinduism-RRB-_~s~_2doesnt exist
QJosé_María_Chacón_~s~_1doesnt exist
QZlatan_Ibrahimović_~s~_10doesnt exist
QThéna

## To train the allennlp 
Go to decomposable_attention.jsonnet file